In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 15.2 MB/s 
     |████████████████████████████████| 895 kB 71.4 MB/s 
     |████████████████████████████████| 596 kB 75.9 MB/s 
     |████████████████████████████████| 6.6 MB 49.2 MB/s 
     |████████████████████████████████| 77 kB 5.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer
from typing import Dict, List
import json

from transformers import (
    EncoderDecoderModel,
    GPT2Tokenizer as BaseGPT2Tokenizer,
    PreTrainedTokenizer,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Trainer
)
from torch.utils.data import Dataset, DataLoader
import torch

In [ ]:
drive_path = '/content/drive/MyDrive/project3/'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

make tokenizer

In [ ]:
limit_alphabet = 4000
vocab_size = 30522

In [ ]:
tokenizer = BertTokenizer(drive_path + f'dump/wpm-vocab-extend-{vocab_size}.txt', do_lower_case=False)

make vocab

In [ ]:
import pickle

def make_corpus(src_file_path, trg_file_path, file_type: str):
    data = []

    if file_type == 'jit':

        with open(src_file_path, 'r') as fd:
                for line in fd.readlines():
                    data.append([line[:-1]])

        with open(trg_file_path, 'r') as fd:
            for i, line in enumerate(fd.readlines()):
                data[i].append(line[:-1])

    elif file_type == 'pickled_ai_hub':

        with open(src_file_path, 'rb') as f:
            tmp_src = pickle.load(f)

        with open(trg_file_path, 'rb') as f:
            tmp_trg = pickle.load(f)
            
        for i in range(len(tmp_src)):
            data.append([tmp_src[i], tmp_trg[i]])
    return data

data split: train, dev, test

In [ ]:
ai_hub_data = make_corpus(drive_path + '전처리된_AI_HUB_data/ai_hub_standard_v2.pkl', drive_path + '전처리된_AI_HUB_data/ai_hub_trans_v2.pkl', 'pickled_ai_hub')
train_ai_hub = ai_hub_data[:-10000]
dev_ai_hub = ai_hub_data[-10000:-5000]
test_ai_hub = ai_hub_data[-5000:]
train_data = make_corpus(drive_path + 'jit/ko.train', drive_path + 'jit/je.train', 'jit')
dev_data = make_corpus(drive_path + 'jit/ko.dev', drive_path + 'jit/je.dev', 'jit')
test_data = make_corpus(drive_path + 'jit/ko.test', drive_path + 'jit/je.test', 'jit')

In [ ]:
len(dev_data)

5000

In [ ]:
ai_hub_data[:10]

[['뭐 어떻게 좀 몰아봤어요 아니면', '뭐 어떵 좀 몰아봐수과 아니면'],
 ['딱 군대 가기 전에 스무살에 제가 군대 갔거든요 그런데', '딱 군대 가기 전에 스무살에 제가 군대 갔거든예 겐디'],
 ['빌려서', '빌려그넹'],
 ['이렇게 서귀포 왔다갔다 하고 바닷가 조금 갔거든요', '이렇게 서귀포 왔다갔다 하고 바닷가 조금 갔거든예'],
 ['근데', '겐디'],
 ['다른 것도 한번 타보려고 했는데 좀 약간 겁도 나고', '다른 것도 한번 타보젠 해신디 좀 약간 겁도 나고'],
 ['한 번은 그 친구 오토바이 좋아해서 친구 오토바이를 이렇게 한번 타봤는데',
  'ᄒᆞᆫ 번은 그 친구 오토바이 좋아해서 친구 오토바이를 이렇게 ᄒᆞᆫ번 타봐신디예'],
 ['저는 그냥 스쿠터 타는 게 제일 좀 좋긴 했거든요', '저는 그냥 스쿠터 타는 게 제일 좀 좋긴 했거든예'],
 ['오토바이 옛날에 군대 갔다 오면 오토바이 하나 사려고 생각해 놨는데',
  '오토바이 옛날에 군대 갔다 오면 오토바이 하나 사젠 생각해 놔신디'],
 ['못사겠어요', '못사크라마시']]

In [ ]:
train_data = train_data + train_ai_hub
dev_data = dev_data + dev_ai_hub
test_data = test_data + test_ai_hub

len(train_data), len(dev_data), len(test_data)

(350356, 10000, 10000)

data tokenized

In [ ]:
def tokenized_data(tokenizer, data, max_length=128, stride=30):
    
    cnt = 0
    embeddings = []
    for src, trg in data:
        src_sample = tokenizer(src, truncation=True, max_length=max_length, stride=stride, return_token_type_ids=False, return_attention_mask=False, return_overflowing_tokens=True)
        trg_sample = tokenizer(trg, truncation=True, max_length=max_length, stride=stride, return_token_type_ids=False, return_attention_mask=False, return_overflowing_tokens=True)
        embeddings.append({'input_ids' : src_sample['input_ids'],
                           'labels': trg_sample['input_ids']})
        if src_sample['num_truncated_tokens'] > 0 and trg_sample['num_truncated_tokens'] > 0:
            src_tmp = src_sample['overflowing_tokens']
            trg_tmp = trg_sample['overflowing_tokens']
            while len(src_tmp) > 0 and len(trg_tmp) > 0:
                cnt += 1
                src_input = [tokenizer.cls_token_id]
                trg_input = [tokenizer.cls_token_id]
                src_input.extend(src_tmp[:max_length-2])
                src_input.append(tokenizer.sep_token_id)
                trg_input.extend(trg_tmp[:max_length-2])
                trg_input.append(tokenizer.sep_token_id)                
                embeddings.append({'input_ids' : src_input,
                               'labels': trg_input})
                src_tmp = src_tmp[max_length-stride-2:2*max_length-stride-2]
                trg_tmp = trg_tmp[max_length-stride-2:2*max_length-stride-2]
    print(f'Processed {cnt} amount of overflowing token set!')

    cnt = 0
    for item in embeddings:
        if len(item['input_ids']) == 2 or len(item['labels']) == 2:
            embeddings.remove(item)
            cnt += 1
    print(f'Removed {cnt} amount of empty token set!')
    return embeddings

In [ ]:
train_tokenized_data = tokenized_data(tokenizer, train_data)
dev_tokenized_data = tokenized_data(tokenizer, dev_data)
test_tokenized_data = tokenized_data(tokenizer, test_data)

Processed 383 amount of overflowing token set!
Removed 48 amount of empty token set!
Processed 17 amount of overflowing token set!
Removed 2 amount of empty token set!
Processed 19 amount of overflowing token set!
Removed 0 amount of empty token set!


In [ ]:
class DatasetRetriever(Dataset):
    def __init__(self, features):
        super(DatasetRetriever, self).__init__()
        self.features = features

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):   
        feature = self.features[index]
        return {
            'input_ids':torch.tensor(feature['input_ids'] ,dtype=torch.long),
            'labels':torch.tensor(feature['labels'] ,dtype=torch.long)
        }

In [ ]:
train_dataset = DatasetRetriever(train_tokenized_data)
dev_dataset = DatasetRetriever(dev_tokenized_data)
test_dataset = DatasetRetriever(test_tokenized_data)

model create

In [ ]:
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel

# Initializing a BERT bert-base-uncased style configuration
config_encoder = BertConfig()
config_decoder = BertConfig()

config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)

# Initializing a Bert2Bert model from the bert-base-uncased style configurations
model = EncoderDecoderModel(config=config)

model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# Accessing the model configuration
config_encoder = model.config.encoder
config_decoder = model.config.decoder
# set decoder config to causal lm
config_encoder.bos_token_id = tokenizer.cls_token_id
config_encoder.eos_token_id = tokenizer.sep_token_id
config_encoder.decoder_start_token_id = tokenizer.cls_token_id
config_decoder.is_decoder = True
config_decoder.add_cross_attention = True
config_decoder.bos_token_id = tokenizer.cls_token_id
config_decoder.eos_token_id = tokenizer.sep_token_id
config_decoder.decoder_start_token_id = tokenizer.cls_token_id

In [ ]:
# pytorch의 randomness 제어를 위한 셀, 최종 모델 생성 시 사용 !!(학습 속도저하됨)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
model.cuda()
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

model train

In [ ]:
collator = DataCollatorForSeq2Seq(tokenizer, model)

arguments = Seq2SeqTrainingArguments(
    output_dir= drive_path + 'dump/models',
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_ratio=0.1,
    gradient_accumulation_steps=1,
    save_total_limit=5,
    dataloader_num_workers=1,
    fp16=True,
    load_best_model_at_end=True
)

trainer = Trainer(
    model,
    arguments,
    data_collator=collator,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset
)

Using amp half precision backend


In [ ]:
trainer.train()

model.save_pretrained(drive_path + "dump/models")

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 350691
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 109600
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:530: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based

Epoch,Training Loss,Validation Loss
1,2.691000,2.638885
2,1.328000,1.488936
3,1.019900,1.233042
4,0.861200,1.136474
5,0.770200,1.074964
6,0.637600,1.052093
7,0.545600,1.042714
8,0.452500,1.051407
9,0.365700,1.072693
10,0.313900,1.084534


***** Running Evaluation *****
  Num examples = 10015
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/project3/dump/models/checkpoint-10960
Configuration saved in /content/drive/MyDrive/project3/dump/models/checkpoint-10960/config.json
Model weights saved in /content/drive/MyDrive/project3/dump/models/checkpoint-10960/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:530: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
***** Running Evaluation *****
  Num examples = 10015
  Batch size = 32
Saving model checkpoint to 

In [ ]:
torch.cuda.empty_cache()

In [ ]:
``